# 🩻 NeuroScan Pro: Multi-Model Anomaly Detection

This notebook sets up and runs the **NeuroScan Pro** environment. It uses **Generative AI (VAE, GAN, ViT)** to detect medical anomalies (Pneumonia) using Unsupervised Learning.

### 🚀 Instructions
1. **Run All Cells** to generate the necessary python files.
2. Ensure you have the **Chest X-Ray Dataset** (Normal/Pneumonia) ready.
3. The final cell will launch the **Streamlit Dashboard**.

In [1]:
# 1. Install Dependencies
!pip install torch torchvision streamlit matplotlib seaborn pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 112.0 MB/s eta 0:00:00


In [2]:
import os
# Create directory for saved models
if not os.path.exists('saved_models'):
    os.makedirs('saved_models')

## 🛠️ Step 1: Define Model Architectures (`model.py`)

In [3]:
%%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F

# ==========================================
# 1. VAE (Variational Autoencoder)
# ==========================================
class MedicalVAE(nn.Module):
    def __init__(self, latent_dim=128):
        super(MedicalVAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, 2, 1), nn.BatchNorm2d(32), nn.LeakyReLU(),
            nn.Conv2d(32, 64, 3, 2, 1), nn.BatchNorm2d(64), nn.LeakyReLU(),
            nn.Conv2d(64, 128, 3, 2, 1), nn.BatchNorm2d(128), nn.LeakyReLU(),
            nn.Conv2d(128, 256, 3, 2, 1), nn.BatchNorm2d(256), nn.LeakyReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(16384, latent_dim)
        self.fc_logvar = nn.Linear(16384, latent_dim)
        self.decoder_input = nn.Linear(latent_dim, 16384)
        self.decoder = nn.Sequential(
            nn.Unflatten(1, (256, 8, 8)),
            nn.ConvTranspose2d(256, 128, 3, 2, 1, 1), nn.BatchNorm2d(128), nn.LeakyReLU(),
            nn.ConvTranspose2d(128, 64, 3, 2, 1, 1), nn.BatchNorm2d(64), nn.LeakyReLU(),
            nn.ConvTranspose2d(64, 32, 3, 2, 1, 1), nn.BatchNorm2d(32), nn.LeakyReLU(),
            nn.ConvTranspose2d(32, 1, 3, 2, 1, 1), nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = self.fc_mu(h), self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        z_proj = self.decoder_input(z)
        return self.decoder(z_proj), mu, logvar

def vae_loss_function(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='sum')
    kld_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld_loss

# ==========================================
# 2. GAN (Generative Adversarial Network)
# ==========================================
class MedicalGANGenerator(nn.Module):
    def __init__(self, latent_dim=128):
        super(MedicalGANGenerator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 3, 2, 1), nn.BatchNorm2d(64), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 3, 2, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(32768, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32768),
            nn.Unflatten(1, (128, 16, 16)),
            nn.ConvTranspose2d(128, 64, 3, 2, 1, 1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, 2, 1, 1), nn.BatchNorm2d(32), nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 3, 2, 1, 1), nn.Sigmoid()
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

class MedicalGANDiscriminator(nn.Module):
    def __init__(self):
        super(MedicalGANDiscriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, 3, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(16, 32, 3, 2, 1), nn.BatchNorm2d(32), nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 3, 2, 1), nn.BatchNorm2d(64), nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(16384, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# ==========================================
# 3. Vision Transformer (ViT-AE)
# ==========================================
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=128, patch_size=16, in_channels=1, embed_dim=128):
        super().__init__()
        self.num_patches = (img_size // patch_size) ** 2
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        x = self.proj(x)
        x = x.flatten(2)
        x = x.transpose(1, 2)
        return x

class MedicalTransformer(nn.Module):
    def __init__(self, img_size=128, patch_size=16, embed_dim=128, depth=4, num_heads=4):
        super().__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, 1, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        self.decoder_proj = nn.Linear(embed_dim, patch_size*patch_size)
        self.patch_size = patch_size
        self.img_size = img_size

    def forward(self, x):
        B, C, H, W = x.shape
        patches = self.patch_embed(x)
        encoded = self.transformer_encoder(patches)
        rec_patches = self.decoder_proj(encoded)
        rec_patches = rec_patches.transpose(1, 2)
        rec_img = F.fold(rec_patches, output_size=(H, W), kernel_size=self.patch_size, stride=self.patch_size)
        return torch.sigmoid(rec_img)

Writing model.py


## 🧪 Step 2: Define Utils (`utils.py`)

In [4]:
%%writefile utils.py
import torch
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score

def calculate_metrics(model, test_loader, threshold_percentile=95):
    model.eval()
    errors = []
    labels = []
    criterion = torch.nn.MSELoss(reduction='none')

    with torch.no_grad():
        for data, label in test_loader:
            data = data.to(next(model.parameters()).device)
            if hasattr(model, 'reparameterize'):
                recon, _, _ = model(data)
            else:
                recon = model(data)
            loss = criterion(recon, data).mean(dim=[1, 2, 3])
            errors.extend(loss.cpu().numpy())
            labels.extend(label.numpy())

    threshold = np.percentile(errors, threshold_percentile)
    preds = [1 if e > threshold else 0 for e in errors]

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='binary')
    return acc, f1

def plot_comparison(original, recons, titles):
    num_models = len(recons)
    cols = num_models + 1
    rows = 2
    fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 8))

    if num_models == 0: return fig
    if len(axes.shape) == 1: axes = axes.reshape(rows, cols)

    orig_np = original.squeeze().cpu().detach().numpy()

    # Row 1: Reconstruction
    axes[0, 0].imshow(orig_np, cmap='gray')
    axes[0, 0].set_title("Original Input", fontsize=14, fontweight='bold')
    axes[0, 0].axis('off')

    for i, (recon, title) in enumerate(zip(recons, titles)):
        recon_np = recon.squeeze().cpu().detach().numpy()
        axes[0, i+1].imshow(recon_np, cmap='gray')
        axes[0, i+1].set_title(f"{title}\n(Reconstruction)", fontsize=12)
        axes[0, i+1].axis('off')

    # Row 2: Anomaly Map
    axes[1, 0].text(0.5, 0.5, "Difference Maps\n(Anomaly Detection)",
                    ha='center', va='center', fontsize=12)
    axes[1, 0].axis('off')

    for i, (recon, title) in enumerate(zip(recons, titles)):
        recon_np = recon.squeeze().cpu().detach().numpy()
        diff_np = np.abs(orig_np - recon_np)
        im = axes[1, i+1].imshow(diff_np, cmap='inferno', vmin=0, vmax=1)
        axes[1, i+1].set_title(f"{title}\n(Anomaly Map)", fontsize=12, color='red')
        axes[1, i+1].axis('off')

    plt.tight_layout()
    return fig

Writing utils.py


## 🖥️ Step 3: Define Streamlit Application (`app.py`)
This file contains the UI logic, Tab structure, and Session State management.

In [5]:
%%writefile app.py
import streamlit as st
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import os
import glob
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt

from model import MedicalVAE, MedicalGANGenerator, MedicalGANDiscriminator, MedicalTransformer, vae_loss_function
from utils import plot_comparison, calculate_metrics

IMAGE_SIZE = 128
BATCH_SIZE = 16
MODEL_DIR = "saved_models"

if not os.path.exists(MODEL_DIR): os.makedirs(MODEL_DIR)

st.set_page_config(page_title="NeuroScan Pro", layout="wide", page_icon="🩻")
st.title="🩻 NeuroScan Pro: Multi-Model Anomaly Detection"

if 'history' not in st.session_state:
    st.session_state['history'] = {'VAE': [], 'GAN': [], 'ViT': []}

if 'leaderboard' not in st.session_state:
    st.session_state['leaderboard'] = []

if 'benchmark_data' not in st.session_state:
    st.session_state['benchmark_data'] = None

if 'dataset_path' not in st.session_state:
    st.session_state['dataset_path'] = os.path.join(os.getcwd(), "chest_xray")

def pick_folder():
    try:
        root = tk.Tk()
        root.withdraw()
        root.wm_attributes('-topmost', 1)
        folder_path = filedialog.askdirectory(master=root)
        root.destroy()
        return folder_path
    except:
        return None

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def get_data_loaders(root_dir, mode='train'):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
    ])
    if not os.path.exists(root_dir): return None, None
    dataset = datasets.ImageFolder(root=root_dir, transform=transform)
    if mode == 'train':
        idx = [i for i, label in enumerate(dataset.targets) if dataset.classes[label] == 'NORMAL']
        subset = Subset(dataset, idx)
        return DataLoader(subset, batch_size=BATCH_SIZE, shuffle=True), dataset.classes
    else:
        return DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False), dataset

def save_model(model, name):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
    path = os.path.join(MODEL_DIR, f"{name}_{timestamp}.pt")
    torch.save(model.state_dict(), path)
    return path

st.sidebar.header="📂 Data & Config"
c1, c2 = st.sidebar.columns([3, 1])
with c2:
    if st.button("📂"):
        fp = pick_folder()
        if fp: st.session_state['dataset_path'] = fp
with c1:
    dataset_path = st.text_input("Dataset Path", value=st.session_state['dataset_path'])

train_path = os.path.join(dataset_path, "train")
test_path = os.path.join(dataset_path, "test")

tab1, tab2 = st.tabs(["🚀 Training & Metrics", "🔎 Comparative Diagnostics"])

with tab1:
    col_ctrl, col_metrics = st.columns([2, 1])
    with col_ctrl:
        st.header("1. Train New Model")
        model_type = st.selectbox("Select Model Architecture", ["VAE", "GAN", "Transformer (ViT)", "Train ALL Sequentially"])
        epochs = st.slider("Epochs", 1, 50, 10)
        lr = st.selectbox("Learning Rate", [1e-3, 1e-4, 2e-4])
        start_btn = st.button("Start Training")

    with col_metrics:
        st.subheader("🏆 Live Leaderboard")
        if st.session_state['leaderboard']:
            st.dataframe(pd.DataFrame(st.session_state['leaderboard']))
            if st.button("Clear Leaderboard"):
                st.session_state['leaderboard'] = []
                st.rerun()
        else:
            st.info("Train a model to see metrics here.")

    if start_btn:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        loader, _ = get_data_loaders(train_path, mode='train')
        test_loader, _ = get_data_loaders(test_path, mode='test')
        if not loader:
            st.error("Could not load training data. Check path.")
            st.stop()

        models_to_train = ["VAE", "GAN", "ViT"] if model_type == "Train ALL Sequentially" else [model_type.split()[0]]
        progress_bar = st.progress(0)

        for m_name in models_to_train:
            st.subheader(f"Training {m_name}...")
            if m_name == "VAE":
                model = MedicalVAE().to(device)
                opt = optim.Adam(model.parameters(), lr=lr)
            elif m_name == "GAN":
                gen = MedicalGANGenerator().to(device)
                disc = MedicalGANDiscriminator().to(device)
                opt_g = optim.Adam(gen.parameters(), lr=lr)
                opt_d = optim.Adam(disc.parameters(), lr=lr)
                criterion = nn.BCELoss()
                model = gen
            elif m_name in ["Transformer", "ViT"]:
                model = MedicalTransformer().to(device)
                opt = optim.Adam(model.parameters(), lr=lr)
                criterion = nn.MSELoss()

            params = count_parameters(model)
            st.info(f"🧠 **{m_name} Parameters:** {params:,}")
            history = []
            for epoch in range(epochs):
                epoch_loss = 0
                for x, _ in loader:
                    x = x.to(device)
                    if m_name == "VAE":
                        opt.zero_grad()
                        recon, mu, logvar = model(x)
                        loss = vae_loss_function(recon, x, mu, logvar)
                        loss.backward()
                        opt.step()
                        epoch_loss += loss.item()
                    elif m_name == "GAN":
                        opt_d.zero_grad()
                        real_labels = torch.ones(x.size(0), 1).to(device)
                        fake_labels = torch.zeros(x.size(0), 1).to(device)
                        d_real = disc(x)
                        d_loss_real = criterion(d_real, real_labels)
                        fake_img = gen(x)
                        d_fake = disc(fake_img.detach())
                        d_loss_fake = criterion(d_fake, fake_labels)
                        d_loss = d_loss_real + d_loss_fake
                        d_loss.backward()
                        opt_d.step()
                        opt_g.zero_grad()
                        d_fake_preds = disc(fake_img)
                        g_adv_loss = criterion(d_fake_preds, real_labels)
                        g_pixel_loss = F.mse_loss(fake_img, x)
                        g_loss = g_adv_loss + (100 * g_pixel_loss)
                        g_loss.backward()
                        opt_g.step()
                        epoch_loss += g_loss.item()
                    elif m_name in ["Transformer", "ViT"]:
                        opt.zero_grad()
                        recon = model(x)
                        loss = criterion(recon, x)
                        loss.backward()
                        opt.step()
                        epoch_loss += loss.item()
                avg_loss = epoch_loss / len(loader)
                history.append(avg_loss)
                progress_bar.progress((epoch + 1) / epochs)
            st.session_state['history'][m_name] = history
            if m_name == "GAN": save_model(gen.cpu(), "GAN")
            else: save_model(model.cpu(), m_name)
            model.cpu()
            acc, f1 = calculate_metrics(model, test_loader)
            st.session_state['leaderboard'].append({
                "Model": m_name,
                "Accuracy": f"{acc:.2%}",
                "F1 Score": f"{f1:.3f}",
                "Params": f"{params:,}",
                "Timestamp": datetime.now().strftime("%H:%M")
            })
            st.success(f"✅ {m_name} Trained & Saved!")

    st.divider()
    st.subheader("Training Performance (Logarithmic Scale)")
    if any(st.session_state['history'].values()):
        fig, ax = plt.subplots()
        for name, hist in st.session_state['history'].items():
            if hist: ax.plot(hist, label=f"{name} Loss")
        ax.set_xlabel("Epochs")
        ax.set_ylabel("Loss (Log Scale)")
        ax.set_yscale('log')
        ax.legend()
        ax.grid(True, which="both", ls="-", alpha=0.5)
        st.pyplot(fig)

with tab2:
    st.header("Comparative Diagnostics")
    saved_files = sorted(glob.glob(os.path.join(MODEL_DIR, "*.pt文")), reverse=True)
    c1, c2, c3 = st.columns(3)
    vae_file = c1.selectbox("Load VAE", [f for f in saved_files if "VAE" in f] + ["None"])
    gan_file = c2.selectbox("Load GAN", [f for f in saved_files if "GAN" in f] + ["None"])
    vit_file = c3.selectbox("Load ViT", [f for f in saved_files if "Transformer" in f or "ViT" in f] + ["None"])
    st.divider()

    def load_selected_models():
        models = {}
        if vae_file != "None":
            m = MedicalVAE()
            m.load_state_dict(torch.load(vae_file, map_location='cpu'))
            models['VAE'] = m
        if gan_file != "None":
            m = MedicalGANGenerator()
            m.load_state_dict(torch.load(gan_file, map_location='cpu'))
            models['GAN'] = m
        if vit_file != "None":
            m = MedicalTransformer()
            m.load_state_dict(torch.load(vit_file, map_location='cpu'))
            models['ViT'] = m
        return models

    st.subheader("1. Model Performance Benchmark")
    if st.button("📊 Run Performance Benchmark", use_container_width=True):
        models = load_selected_models()
        if not models:
            st.error("Please load at least one model.")
        else:
            test_loader, _ = get_data_loaders(test_path, mode='test')
            with st.spinner("Calculating Metrics..."):
                metrics_data = []
                for name, model in models.items():
                    acc, f1 = calculate_metrics(model, test_loader)
                    params = count_parameters(model)
                    metrics_data.append({"Model": name, "Accuracy": f"{acc:.2%}", "F1 Score": f"{f1:.3f}", "Parameters": f"{params:,}"})
                st.session_state['benchmark_data'] = pd.DataFrame(metrics_data)

    if st.session_state['benchmark_data'] is not None:
        st.table(st.session_state['benchmark_data'])
        if st.button("Clear Table"):
            st.session_state['benchmark_data'] = None
            st.rerun()

    st.divider()
    st.subheader("2. Visual Reconstruction Inspection")
    if st.button("👁️ Run Visual Inspection (Random Batch)", use_container_width=True):
        models = load_selected_models()
        if not models:
            st.error("Please load at least one model.")
        else:
            _, full_test_dataset = get_data_loaders(test_path, mode='test')
            def get_random_image(target_label_name):
                target_idx = full_test_dataset.class_to_idx[target_label_name]
                indices = [i for i, label in enumerate(full_test_dataset.targets) if label == target_idx]
                rand_idx = np.random.choice(indices)
                img, _ = full_test_dataset[rand_idx]
                return img.unsqueeze(0)

            col_normal, col_pneumonia = st.columns(2)
            with col_normal:
                st.info("🟢 **Control Case: NORMAL**")
                img_normal = get_random_image("NORMAL")
                recons_n = []
                titles_n = []
                for name, model in models.items():
                    model.eval()
                    with torch.no_grad():
                        recon = model(img_normal) if name != "VAE" else model(img_normal)[0]
                    recons_n.append(recon)
                    titles_n.append(name)
                fig_n = plot_comparison(img_normal, recons_n, titles_n)
                st.pyplot(fig_n)

            with col_pneumonia:
                st.error("🔴 **Test Case: PNEUMONIA**")
                img_pneu = get_random_image("PNEUMONIA")
                recons_p = []
                titles_p = []
                for name, model in models.items():
                    model.eval()
                    with torch.no_grad():
                        recon = model(img_pneu) if name != "VAE" else model(img_pneu)[0]
                    recons_p.append(recon)
                    titles_p.append(name)
                fig_p = plot_comparison(img_pneu, recons_p, titles_p)
                st.pyplot(fig_p)
            st.caption("Each click loads a new random set of images.")

Writing app.py


## 🚀 Step 4: Run the Dashboard
Run the following command to start the app. It will provide a local URL.

In [8]:
# 1. Install localtunnel
!npm install localtunnel

# 2. Get the Password (IP Address)
# IMPORTANT: You will need to enter this IP on the website that opens.
import urllib
print("Password/Endpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# 3. Run Streamlit in the background & Expose it
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸Password/Endpoint IP for localtunnel is: 35.187.239.97
⠙⠹

your url is: https://angry-insects-mate.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.187.239.97:8501

  Stopping...
^C
